In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import KernelPCA

from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt


def apply_kernel_pca(indep_X, n_components=6, kernel='rbf'):
    from sklearn.decomposition import KernelPCA
    kpca = KernelPCA(n_components=n_components, kernel=kernel)
    X_kpca = kpca.fit_transform(indep_X)  # Perform Kernel PCA on the passed input
    return X_kpca


def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    
 
def cm_prediction(classifier,X_test):
         y_pred = classifier.predict(X_test)
            
            # Making the Confusion Matrix
         from sklearn.metrics import confusion_matrix
         cm = confusion_matrix(y_test, y_pred)
            
         from sklearn.metrics import accuracy_score 
         from sklearn.metrics import classification_report 
            #from sklearn.metrics import confusion_matrix
            #cm = confusion_matrix(y_test, y_pred)
            
         Accuracy=accuracy_score(y_test, y_pred )
            
         report=classification_report(y_test, y_pred)
         return  classifier,Accuracy,report,X_test,y_test,cm

def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      
    
def svm_linear(X_train,y_train,X_test):
            from sklearn.svm import SVC
            classifier = SVC(kernel = 'linear', random_state = 0)
            classifier.fit(X_train, y_train)
            classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
            return  classifier,Accuracy,report,X_test,y_test,cm
    
def svm_NL(X_train,y_train,X_test):
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
   
def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm         
    
    
def knn(X_train,y_train,X_test):
           
        # Fitting K-NN to the Training set
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      


def random(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def principalAnalysis(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf): 
    
    dataframe=pd.DataFrame(index=['ChiSquare'],columns=['Logistic','SVMl','SVMnl','KNN','Navie','Decision','Random'])
    for number,idex in enumerate(dataframe.index):      
        dataframe['Logistic'][idex]=acclog[number]       
        dataframe['SVMl'][idex]=accsvml[number]
        dataframe['SVMnl'][idex]=accsvmnl[number]
        dataframe['KNN'][idex]=accknn[number]
        dataframe['Navie'][idex]=accnav[number]
        dataframe['Decision'][idex]=accdes[number]
        dataframe['Random'][idex]=accrf[number]
    return dataframe

In [2]:

dataset1 = pd.read_csv("PreProcessedEmployee.csv", index_col=None)
dataset1
df2 = dataset1
df2 = pd.get_dummies(df2, drop_first=True)


# Convert all boolean columns to 0 and 1, except 'Resigned'
for col in df2.select_dtypes(include=['bool']).columns:
    if col != 'Resigned':  # Keep 'Resigned' as is
        df2[col] = df2[col].astype(int)
df2
indep_X = df2.drop(['Performance_Score','Work_Hours_Per_Week','Projects_Handled','Resigned','Department_Engineering',	'Department_Finance','Gender_Other','Job_Title_Consultant',	'Job_Title_Developer',	'Job_Title_Engineer',	'Job_Title_Manager'	,'Job_Title_Specialist'	,'Job_Title_Technician',	'Education_Level_Master',	'Education_Level_PhD'] ,axis=1)  # Features
dep_Y = df2['Performance_Score']  # Target


In [3]:
print(df2.columns)

Index(['Age', 'Years_At_Company', 'Performance_Score', 'Monthly_Salary',
       'Work_Hours_Per_Week', 'Projects_Handled', 'Promotions', 'Resigned',
       'Department_Engineering', 'Department_Finance', 'Department_HR',
       'Department_IT', 'Department_Legal', 'Department_Marketing',
       'Department_Operations', 'Department_Sales', 'Gender_Male',
       'Gender_Other', 'Job_Title_Consultant', 'Job_Title_Developer',
       'Job_Title_Engineer', 'Job_Title_Manager', 'Job_Title_Specialist',
       'Job_Title_Technician', 'Education_Level_High School',
       'Education_Level_Master', 'Education_Level_PhD'],
      dtype='object')


In [4]:
df2

,Age,Years_At_Company,Performance_Score,Monthly_Salary,Work_Hours_Per_Week,Projects_Handled,Promotions,Resigned,Department_Engineering,Department_Finance,...,Gender_Other,Job_Title_Consultant,Job_Title_Developer,Job_Title_Engineer,Job_Title_Manager,Job_Title_Specialist,Job_Title_Technician,Education_Level_High School,Education_Level_Master,Education_Level_PhD
0,55,2,5.0,6750,33,32,0,False,0,0,...,0,0,0,0,0,1,0,1,0,0
1,29,0,5.0,7500,34,34,2,False,0,1,...,0,0,1,0,0,0,0,1,0,0
2,55,8,3.0,5850,37,27,0,False,0,1,...,0,0,0,0,0,1,0,1,0,0
3,48,7,2.0,4800,52,10,1,False,0,0,...,0,0,0,0,0,0,0,0,0,0
4,36,3,2.0,4800,38,11,1,False,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,45,4,5.0,9000,48,9,1,False,0,0,...,0,0,0,0,1,0,0,0,0,0
245,32,2,4.0,6300,55,46,2,False,0,0,...,0,0,0,0,0,1,0,0,0,0
246,24,7,4.0,4900,30,33,0,False,0,0,...,0,0,0,0,0,0,1,0,0,0
247,43,3,2.0,4800,57,36,1,False,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Apply PCA to the dataset
kernel_pca_data = apply_kernel_pca(indep_X, n_components=5, kernel='rbf')


acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]


In [15]:
kernel_pca_data

array([[-0.00911871, -0.01349052,  0.00215046, -0.01516782, -0.00838028],
       [-0.009999  , -0.01488897,  0.00242611, -0.01717577, -0.0096665 ],
       [-0.01249685, -0.01896272,  0.00330885, -0.02371609, -0.01424467],
       ...,
       [-0.00854576, -0.01257769,  0.0019689 , -0.01384337, -0.00752631],
       [ 0.73600474,  0.16319724, -0.00673437,  0.04217049,  0.01401963],
       [-0.01019599, -0.01520652,  0.00249189, -0.017659  , -0.00998932]])

In [16]:
X_train, X_test, y_train, y_test = split_scalar(kernel_pca_data, dep_Y)   
        
classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
acclog.append(Accuracy)

classifier,Accuracy,report,X_test,y_test,cm=svm_linear(X_train,y_train,X_test)  
accsvml.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
accsvmnl.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=knn(X_train,y_train,X_test)  
accknn.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Navie(X_train,y_train,X_test)  
accnav.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)  
accdes.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
accrf.append(Accuracy)
    
result=principalAnalysis(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf)

C:\Users\sindhiya maria\anaconda3\envs\Aiml\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sindhiya maria\anaconda3\envs\Aiml\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sindhiya maria\anaconda3\envs\Aiml\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [22]:
result
#6

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.206349,0.285714,0.253968,0.301587,0.15873,0.380952,0.396825


In [26]:
result
#4

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.222222,0.253968,0.238095,0.301587,0.142857,0.349206,0.31746


In [8]:
result
#2

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.253968,0.253968,0.238095,0.301587,0.15873,0.301587,0.349206


In [13]:
result
#3

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.253968,0.253968,0.238095,0.253968,0.126984,0.349206,0.31746


In [17]:
result
#5

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.238095,0.253968,0.238095,0.285714,0.142857,0.333333,0.365079


Kernel_PCA using "Poly"

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import KernelPCA

from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt


def apply_kernel_pca(indep_X, n_components=2, kernel='poly'):
    from sklearn.decomposition import KernelPCA
    kpca = KernelPCA(n_components=n_components, kernel=kernel)
    X_kpca = kpca.fit_transform(indep_X)  # Perform Kernel PCA on the passed input
    return X_kpca


def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    
 
def cm_prediction(classifier,X_test):
         y_pred = classifier.predict(X_test)
            
            # Making the Confusion Matrix
         from sklearn.metrics import confusion_matrix
         cm = confusion_matrix(y_test, y_pred)
            
         from sklearn.metrics import accuracy_score 
         from sklearn.metrics import classification_report 
            #from sklearn.metrics import confusion_matrix
            #cm = confusion_matrix(y_test, y_pred)
            
         Accuracy=accuracy_score(y_test, y_pred )
            
         report=classification_report(y_test, y_pred)
         return  classifier,Accuracy,report,X_test,y_test,cm

def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      
    
def svm_linear(X_train,y_train,X_test):
            from sklearn.svm import SVC
            classifier = SVC(kernel = 'linear', random_state = 0)
            classifier.fit(X_train, y_train)
            classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
            return  classifier,Accuracy,report,X_test,y_test,cm
    
def svm_NL(X_train,y_train,X_test):
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
   
def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm         
    
    
def knn(X_train,y_train,X_test):
           
        # Fitting K-NN to the Training set
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      


def random(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def principalAnalysis(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf): 
    
    dataframe=pd.DataFrame(index=['ChiSquare'],columns=['Logistic','SVMl','SVMnl','KNN','Navie','Decision','Random'])
    for number,idex in enumerate(dataframe.index):      
        dataframe['Logistic'][idex]=acclog[number]       
        dataframe['SVMl'][idex]=accsvml[number]
        dataframe['SVMnl'][idex]=accsvmnl[number]
        dataframe['KNN'][idex]=accknn[number]
        dataframe['Navie'][idex]=accnav[number]
        dataframe['Decision'][idex]=accdes[number]
        dataframe['Random'][idex]=accrf[number]
    return dataframe

In [19]:

dataset1 = pd.read_csv("PreProcessedEmployee.csv", index_col=None)
dataset1
df2 = dataset1
df2 = pd.get_dummies(df2, drop_first=True)


# Convert all boolean columns to 0 and 1, except 'Resigned'
for col in df2.select_dtypes(include=['bool']).columns:
    if col != 'Resigned':  # Keep 'Resigned' as is
        df2[col] = df2[col].astype(int)
df2
indep_X = df2.drop(['Performance_Score','Work_Hours_Per_Week','Projects_Handled','Resigned','Department_Engineering',	'Department_Finance','Gender_Other','Job_Title_Consultant',	'Job_Title_Developer',	'Job_Title_Engineer',	'Job_Title_Manager'	,'Job_Title_Specialist'	,'Job_Title_Technician',	'Education_Level_Master',	'Education_Level_PhD'] ,axis=1)  # Features
dep_Y = df2['Performance_Score']  # Target


In [20]:
df2

,Age,Years_At_Company,Performance_Score,Monthly_Salary,Work_Hours_Per_Week,Projects_Handled,Promotions,Resigned,Department_Engineering,Department_Finance,...,Gender_Other,Job_Title_Consultant,Job_Title_Developer,Job_Title_Engineer,Job_Title_Manager,Job_Title_Specialist,Job_Title_Technician,Education_Level_High School,Education_Level_Master,Education_Level_PhD
0,55,2,5.0,6750,33,32,0,False,0,0,...,0,0,0,0,0,1,0,1,0,0
1,29,0,5.0,7500,34,34,2,False,0,1,...,0,0,1,0,0,0,0,1,0,0
2,55,8,3.0,5850,37,27,0,False,0,1,...,0,0,0,0,0,1,0,1,0,0
3,48,7,2.0,4800,52,10,1,False,0,0,...,0,0,0,0,0,0,0,0,0,0
4,36,3,2.0,4800,38,11,1,False,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,45,4,5.0,9000,48,9,1,False,0,0,...,0,0,0,0,1,0,0,0,0,0
245,32,2,4.0,6300,55,46,2,False,0,0,...,0,0,0,0,0,1,0,0,0,0
246,24,7,4.0,4900,30,33,0,False,0,0,...,0,0,0,0,0,0,1,0,0,0
247,43,3,2.0,4800,57,36,1,False,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# Apply PCA to the dataset
kernel_pca_data = apply_kernel_pca(indep_X, n_components=5, kernel='poly')



acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]


In [24]:
kernel_pca_data

array([[ 4.76341620e+08, -3.12879030e+07, -5.85390249e+06,
         1.86007404e+06,  1.59176175e+06],
       [ 3.22634392e+09,  2.36586395e+07, -9.22301551e+06,
        -1.86783839e+06,  1.48931344e+06],
       [-2.10609357e+09, -2.29119869e+07,  4.05483842e+06,
         1.06988994e+06,  1.02448559e+06],
       ...,
       [-4.09232204e+09,  1.84077685e+07,  3.06343716e+06,
         9.40090082e+05, -5.92315628e+05],
       [-4.26197149e+09,  2.23539111e+05, -1.84435414e+06,
         5.97784616e+04, -7.87649999e+05],
       [ 1.87082832e+09,  1.41982536e+07, -4.24702516e+06,
         2.25479782e+06,  1.57099695e+06]])

In [25]:
X_train, X_test, y_train, y_test = split_scalar(kernel_pca_data, dep_Y)   
        
classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
acclog.append(Accuracy)

classifier,Accuracy,report,X_test,y_test,cm=svm_linear(X_train,y_train,X_test)  
accsvml.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
accsvmnl.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=knn(X_train,y_train,X_test)  
accknn.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Navie(X_train,y_train,X_test)  
accnav.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)  
accdes.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
accrf.append(Accuracy)
    
result=principalAnalysis(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf)


C:\Users\sindhiya maria\anaconda3\envs\Aiml\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sindhiya maria\anaconda3\envs\Aiml\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sindhiya maria\anaconda3\envs\Aiml\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [34]:
result
#2

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.301587,0.365079,0.333333,0.238095,0.31746,0.761905,0.47619


In [42]:
result
#3

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.253968,0.253968,0.301587,0.190476,0.285714,0.730159,0.365079


In [38]:
result
#4

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.253968,0.285714,0.222222,0.238095,0.285714,0.730159,0.539683


In [46]:
result
#6

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.206349,0.269841,0.253968,0.31746,0.222222,0.714286,0.31746


In [26]:
result
#5

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.238095,0.253968,0.285714,0.285714,0.269841,0.698413,0.333333
